# Experimento: C3 - Mondial - Foreign Key - GPT 3.5

In [ ]:

from langchain.chat_models import ChatOpenAI
from urllib.parse import quote  

from dotenv import load_dotenv
import time
import os
import sys
import json
load_dotenv()

from c3_clear_prompting import generate_clear_prompting
from c3_calibration_with_hints import generate_calibration_with_hints
from c3_generate_sql import generate_sql

experiment_path = '..\..'
path = os.path.abspath('')
module_path = os.path.join(path, experiment_path)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils

In [2]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'
FILE_NAME_RESULT = f"results/5_c3_queries_chatgpt_{SCHEMA}_fk.json"

In [3]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 
        
def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries


## Conexão com o banco

In [ ]:
json_file_path = f"{experiment_path}/datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)
db_connection

In [ ]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables, schema=db_connection.get("SCHEMA", None))
db.get_table_names()

## C3 - Function

In [6]:
def run_c3(question, db, model='gpt-3.5-turbo', add_fk = True, callback= None):
    llm = ChatOpenAI(model_name = model, temperature=0.7, n=10)
    clear_prompting = generate_clear_prompting(question, db, llm, add_fk=add_fk, callback=callback)
    print(clear_prompting)
    messages = generate_calibration_with_hints(clear_prompting)
    llm = ChatOpenAI(model_name = model, n=20)
    sql = generate_sql(messages, llm, db, question, callback=callback)
    return sql

## Preparando as consultas 

In [7]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

## Tracking token usage

In [8]:
track_token = [] 
def tracking_token(cb =None, reset = False):
    global track_token
    track_token.append(cb)
    if reset:
        track_token = []

In [9]:

def convert_to_dict_tracking_token():
    token_usage = {}
    for e in track_token:
        for key in e.keys():
            token_usage[key] = {}
            token_usage[key]['total_tokens'] = e[key].total_tokens
            token_usage[key]['total_cost'] = e[key].total_cost
            token_usage[key]['prompt_tokens'] = e[key].prompt_tokens
            token_usage[key]['completion_tokens'] = e[key].completion_tokens 
    return token_usage

In [ ]:
sql = run_c3("'What are the languages spoken in Poland?", db, callback=tracking_token)
print(convert_to_dict_tracking_token())

In [11]:
print(sql)
track_token


SELECT language.name FROM language INNER JOIN country ON language.country = country.code WHERE country.name = 'Poland';


[{'table_recall': Tokens Used: 2760
  	Prompt Tokens: 890
  	Completion Tokens: 1870
  Successful Requests: 1
  Total Cost (USD): $0.0050750000000000005},
 {'column_recall': Tokens Used: 3443
  	Prompt Tokens: 238
  	Completion Tokens: 3205
  Successful Requests: 1
  Total Cost (USD): $0.006767},
 {'sql_generation': Tokens Used: 834
  	Prompt Tokens: 506
  	Completion Tokens: 328
  Successful Requests: 1
  Total Cost (USD): $0.001415}]

## Executando o método

In [12]:
tracking_token(reset=True)
errors = []
query_index = -1
for instance in queries:
    query_index +=1
    try:
        start_time = time.time()
        sql = run_c3(instance["question"], db, callback=tracking_token)
        end_time = time.time()
        instance["query_string"] = sql
        instance["token_usage"] = convert_to_dict_tracking_token()
        instance['time'] = end_time - start_time
        save_queries(queries)
        print(instance['id'], instance['question'], instance["query_string"], instance['time'])
        print("-----")
    except:
        print("Error")
        errors.append(query_index)
        pass
    finally:
        tracking_token(reset=True)




1 - Ranking the columns in each table based on relevance to the question:

Table: continent
1. name - This column is relevant because it contains the name of the continent. Although not directly related to the question, it could be used to join tables.
2. area - This column is highly relevant as it directly provides the area of the continent.

Table: country
1. name - This column is relevant as it contains the name of the country. It could be used to join tables.
2. code - This column is highly relevant as it is the primary key of the country table and can be used to join tables.
3. capital - This column is relevant as it contains the name of the capital city of the country. It could be used to join tables.
4. province - This column is relevant as it contains the name of the province. It could be used to join tables.
5. area - This column is relevant as it directly provides the area of the country.
6. population - This column is relevant as it provides the population of the country.

T

In [14]:
errors

[]

#### Fixing query

In [12]:
pos = 40
instance = queries[pos]
q = read_queries()
start_time = time.time()
sql = run_c3(instance["question"], db, callback=tracking_token)
end_time = time.time()
instance["query_string"] = sql
instance["token_usage"] = convert_to_dict_tracking_token()
instance['time'] = end_time - start_time
q[pos] = instance
save_queries(q)
print(instance['id'], instance['question'], instance["query_string"], instance['time'])



Column recall attempt: 1

### Complete oracle SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query. 
### Oracle SQL tables, with their properties: 
#
# country (name, code, capital, province)
# sea (name, area, depth)
# borders (country1, country2, length)
# borders.country1=country.code
# borders.country2=country.code

#
### How many countries that are close to the Mediterranean Sea?
SELECT
41 How many countries that are close to the Mediterranean Sea? SELECT COUNT(DISTINCT country.name) FROM country JOIN borders ON country.code = borders.country1 JOIN sea ON borders.country2 = sea.name WHERE sea.name = 'Mediterranean Sea'; 42.66082406044006


In [ ]:
to_fix = [59,62,72,85,99]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    start_time = time.time()
    sql = run_c3(instance["question"], db, callback=tracking_token)
    end_time = time.time()
    instance["query_string"] = sql
    instance["token_usage"] = convert_to_dict_tracking_token()
    instance['time'] = end_time - start_time
    q[pos] = instance
    save_queries(q)
    print(instance['id'], instance['question'], instance["query_string"], instance['time'])

In [15]:
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area FROM country WHERE name = 'Thailand';",
  'type': 'simple',
  'token_usage': {'table_recall': {'total_tokens': 3784,
    'total_cost': 0.0072075,
    'prompt_tokens': 721,
    'completion_tokens': 3063},
   'column_recall': {'total_tokens': 5752,
    'total_cost': 0.011373499999999998,
    'prompt_tokens': 261,
    'completion_tokens': 5491},
   'sql_generation': {'total_tokens': 798,
    'total_cost': 0.001329,
    'prompt_tokens': 534,
    'completion_tokens': 264}}},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT province.name FROM province WHERE province.area > 10000;',
  'type': 'simple',
  'token_usage': {'table_recall': {'total_tokens': 2289,
    'total_cost': 0.0042144999999999995,
    'prompt_tokens': 727,
    'completion_tokens': 1562},
   'column_recall': {'total_tokens': 4013,
    'total_cost': 0.0079085,
    'prompt_tok